## Start and prep

In [ ]:
# we will use Kaggle dataset from here:
# https://www.kaggle.com/datasets/blastchar/telco-customer-churn
# code and video from here:
# https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/03-classification/01-churn-project.md
# notebook 
# https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/03-classification/notebook.ipynb
# dataset for download
# https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv

# Data preparation

# Download the data, read it with pandas
# Look at the data
# Make column names and values look uniform
# Check if all the columns read correctly
# Check if the churn variable needs any preparation
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'
